In [1]:
!git clone https://github.com/nkt780426/Bachelor-s-Project.git

Cloning into 'Bachelor-s-Project'...
remote: Enumerating objects: 541, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 541 (delta 33), reused 101 (delta 28), pack-reused 408 (from 1)
Receiving objects: 100% (541/541), 77.04 MiB | 43.92 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [2]:
!mv Bachelor-s-Project/going_modular .

In [3]:
!ls -la

total 48
drwxr-xr-x 4 root root  4096 Dec 20 06:11 .
drwxr-xr-x 6 root root  4096 Dec 20 06:10 ..
drwxr-xr-x 5 root root  4096 Dec 20 06:11 Bachelor-s-Project
drwxr-xr-x 8 root root  4096 Dec 20 06:11 going_modular
---------- 1 root root 30183 Dec 20 06:11 __notebook__.ipynb


In [4]:
import os

import torch
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A

from going_modular.dataloader.magface import create_magface_dataloader
from going_modular.model.MagFaceRecognition import MagFaceRecognition
from going_modular.train_eval.magface.train import fit
from going_modular.loss.MagLoss import MagLoss
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    # Thư mục
    'type': 'normalmap',
    'train_dir': '/kaggle/input/hoangvn-3dfacerecognition/Normal_Map/train',
    'test_dir': '/kaggle/input/hoangvn-3dfacerecognition/Normal_Map/test',
    
    # Cấu hình train
    'backbone': 'inception-resnet-v1',
    'epochs': 500,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 224,
    'num_class': len(os.listdir('/kaggle/input/hoangvn-3dfacerecognition/Normal_Map/train')),
    'embedding_size': 512,
    
    'learning_rate': 0.2,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # Hàm m(ai) giúp thay đổi ai từ 0.25 đến 1.6
    'scale': 64,
    'lambda_g': 20,
    'l_margin': 0.45, 
    'u_margin': 0.8,
    'l_a': 10, 
    'u_a': 110,
}

train_transform = A.Compose([
    RandomResizedCropRect(256),
    GaussianNoise(),
])

test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
])

train_dataloader, test_dataloader = create_magface_dataloader(CONFIGURATION, train_transform, test_transform)

model = MagFaceRecognition(CONFIGURATION).to(device)
criterion = MagLoss(conf = CONFIGURATION)

optimizer = torch.optim.Adam(model.parameters(), lr=CONFIGURATION['learning_rate'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=40, T_mult=1, eta_min=1e-6)
checkpoint_path = os.path.abspath('checkpoint/magface/' + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)
earlystop_dir = os.path.abspath('checkpoint/magface/' + CONFIGURATION['type'] + '/models')
early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['test_cosine_auc', 'test_cosine_accuracy', 'test_euclidean_auc', 'test_euclidean_accuracy'],
    patience=50,
    mode='max',
    verbose=1,
    save_dir=earlystop_dir,
    start_from_epoch=50
)            

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [5]:
fit(CONFIGURATION, 0, model, device, train_dataloader, test_dataloader, criterion, optimizer, scheduler, early_stopping, modle_checkpoint)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1:
	train: loss 2.783 | loss id   2.76 | top_1_acc 0.0045 | top_5_acc 0.0281 | cos_auc: 0.5634 | cos_acc: 0.6764 | eu_auc: 0.5509 | eu_acc: 0.8130
	test: cos_auc: 0.5589 | cos_acc: 0.7392 | eu_auc: 0.5469 | eu_acc: 0.7977
	Saving model and optimizer state to /kaggle/working/checkpoint/magface/normalmap/models/checkpoint.pth


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 2:
	train: loss 0.625 | loss id   0.60 | top_1_acc 0.0056 | top_5_acc 0.0259 | cos_auc: 0.6181 | cos_acc: 0.6245 | eu_auc: 0.5973 | eu_acc: 0.5910
	test: cos_auc: 0.6528 | cos_acc: 0.7008 | eu_auc: 0.6393 | eu_acc: 0.7573
	Saving model and optimizer state to /kaggle/working/checkpoint/magface/normalmap/models/checkpoint.pth


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

Epoch 455:
	train: loss 0.476 | loss id   0.45 | top_1_acc 0.9966 | top_5_acc 0.9989 | cos_auc: 1.0000 | cos_acc: 0.9994 | eu_auc: 0.9993 | eu_acc: 0.9923
	test: cos_auc: 0.8564 | cos_acc: 0.8275 | eu_auc: 0.8009 | eu_acc: 0.7584
	Saving model and optimizer state to /kaggle/working/checkpoint/magface/normalmap/models/checkpoint.pth
	Epoch 455: EarlyStopping counter for 'test_cosine_auc': 48/50
	Epoch 455: EarlyStopping counter for 'test_cosine_accuracy': 238/50
	Epoch 455: EarlyStopping counter for 'test_euclidean_auc': 173/50
	Epoch 455: EarlyStopping counter for 'test_euclidean_accuracy': 201/50


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 456:
	train: loss 0.408 | loss id   0.39 | top_1_acc 1.0000 | top_5_acc 1.0000 | cos_auc: 1.0000 | cos_acc: 1.0000 | eu_auc: 0.9994 | eu_acc: 0.9911
	test: cos_auc: 0.8720 | cos_acc: 0.8598 | eu_auc: 0.8117 | eu_acc: 0.7593
	Saving model and optimizer state to /kaggle/working/checkpoint/magface/normalmap/models/checkpoint.pth
	Epoch 456: EarlyStopping counter for 'test_cosine_auc': 49/50
	Epoch 456: EarlyStopping counter for 'test_cosine_accuracy': 239/50
	Epoch 456: EarlyStopping counter for 'test_euclidean_auc': 174/50
	Epoch 456: EarlyStopping counter for 'test_euclidean_accuracy': 202/50


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 457:
	train: loss 0.403 | loss id   0.38 | top_1_acc 0.9989 | top_5_acc 1.0000 | cos_auc: 1.0000 | cos_acc: 1.0000 | eu_auc: 0.9998 | eu_acc: 0.9959
	test: cos_auc: 0.8678 | cos_acc: 0.8214 | eu_auc: 0.8102 | eu_acc: 0.6700
	Saving model and optimizer state to /kaggle/working/checkpoint/magface/normalmap/models/checkpoint.pth
	Epoch 457: EarlyStopping counter for 'test_cosine_auc': 50/50
	Epoch 457: EarlyStopping counter for 'test_cosine_accuracy': 240/50
	Epoch 457: EarlyStopping counter for 'test_euclidean_auc': 175/50
	Epoch 457: EarlyStopping counter for 'test_euclidean_accuracy': 203/50
	Early stopping triggered at epoch 457 as all metrics exceeded patience.
